- armar un df por product/client 

- identificar los productos con menos de 12 meses de hostoria
    - guardar el promedio de vtas por client/producto, para esos productos, en un df "predicciones"
    - sacar las vtas de esos productos del df

- para los productos que quedan, re-calcular las vtas de agosto como el promedio de julio/sept (esto por producto/client)
- convertir las categoricas a numericas
- aplicar los scalers

- aplicar Dynamic Time Wrapping

#### Imports

La idea de esta variante, es predecir solamente los productos que tengan mas de 12 meses de datos (usando los ultimos 12 solamente), y aquellos con menos datos, predecir la media

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from skopt import BayesSearchCV
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from scikeras.wrappers import KerasRegressor
from keras.callbacks import EarlyStopping
import numpy as np
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.metrics import MeanSquaredError
from keras.regularizers import l2
from keras.layers import BatchNormalization
from skopt.space import Categorical
from sklearn.metrics import mean_squared_error

2024-06-19 15:35:27.087821: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
final_dataset = pd.read_csv('../../Datasets/final_dataset_descr.csv', sep='\t')

In [3]:
final_dataset.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age
0,201701,10001,20001,0,11,99.43861,99.43861,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
1,201701,10002,20001,0,17,38.68301,35.72806,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
2,201701,10003,20001,0,17,143.49426,143.49426,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
3,201701,10004,20001,0,9,184.72927,184.72927,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
4,201701,10005,20001,0,23,19.08407,19.08407,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0


#### Pre procesamiento

In [4]:
treshold_months = 12
training_trashold = treshold_months + 2 

In [5]:
######
# Para los productos que tengan menos de 12 meses de historia, predigo (por producto/cliente), la media de ventas para ese producto/cliente
######
history_count = final_dataset.groupby(['customer_id', 'product_id'])['periodo'].nunique()
combinations_less_than_treshold_months = history_count[history_count < training_trashold].index

predicciones = final_dataset.set_index(['customer_id', 'product_id']).loc[combinations_less_than_treshold_months].reset_index()
promedio_ventas = predicciones.groupby(['customer_id', 'product_id']).agg(
    promedio_tn=('tn', 'mean')
).reset_index()

final_dataset_filtered = final_dataset.set_index(['customer_id', 'product_id']).drop(index=combinations_less_than_treshold_months).reset_index()
predicciones = promedio_ventas

In [6]:
display(predicciones)
display(final_dataset_filtered)

,customer_id,product_id,promedio_tn
0,10001,20010,2.274273
1,10001,20021,1.150656
2,10001,20041,2.044225
3,10001,20130,2.752506
4,10001,20143,2.006005
...,...,...,...
201934,10636,20282,0.019460
201935,10636,20386,0.008390
201936,10636,20838,0.001900
201937,10637,20657,0.001440


,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age
0,10001,20001,201701,0,11,99.43861,99.43861,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
1,10002,20001,201701,0,17,38.68301,35.72806,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
2,10003,20001,201701,0,17,143.49426,143.49426,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
3,10004,20001,201701,0,9,184.72927,184.72927,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
4,10005,20001,201701,0,23,19.08407,19.08407,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320495,10018,21105,201908,0,3,0.01652,0.01652,PC,CABELLO,ACONDICIONADOR,SHAMPOO3,170,hidratacion,Q3,8,0,9
1320496,10018,21105,201909,0,2,0.00733,0.00733,PC,CABELLO,ACONDICIONADOR,SHAMPOO3,170,hidratacion,Q3,9,1,10
1320497,10018,21105,201910,0,5,0.01286,0.01286,PC,CABELLO,ACONDICIONADOR,SHAMPOO3,170,hidratacion,Q4,10,0,11
1320498,10018,21105,201911,0,1,0.05137,0.05137,PC,CABELLO,ACONDICIONADOR,SHAMPOO3,170,hidratacion,Q4,11,0,12


In [7]:
######
# Promedio las ventras de Agosto del 2019, como las de Julio y Septiembre, dado que este mes
# la multinacional decidio dejar de vender algunos dias por la crisis, lo cual representa un outlier fuerte.
######

july_2019 = final_dataset_filtered[final_dataset_filtered['periodo'] == '201907']
september_2019 = final_dataset_filtered[final_dataset_filtered['periodo'] == '201909']

# Renombramos las columnas 'tn' para diferenciarlas al hacer el merge
july_2019 = july_2019.rename(columns={'tn': 'tn_july'})
september_2019 = september_2019.rename(columns={'tn': 'tn_september'})

# Hacemos un merge de los dos dataframes sobre 'customer_id' y 'product_id'
merged = pd.merge(july_2019, september_2019, on=['customer_id', 'product_id'], suffixes=('_july', '_september'))

# Calculamos el promedio de 'tn' para agosto de 2019
merged['tn_august'] = merged[['tn_july', 'tn_september']].mean(axis=1)

# Creamos un nuevo dataframe para agosto de 2019 con las columnas necesarias
august_2019 = merged[['customer_id', 'product_id', 'tn_august']].copy()
august_2019['periodo'] = pd.to_datetime('20190801', format='%Y%m%d')
august_2019 = august_2019.rename(columns={'tn_august': 'tn'})

# Añadimos las columnas adicionales necesarias para coincidir con el dataframe original
additional_columns = final_dataset_filtered.columns.difference(['tn', 'periodo', 'customer_id', 'product_id'])
for col in additional_columns:
    august_2019[col] = merged[f"{col}_july"]

# Concatenamos el nuevo dataframe con el dataframe original filtrado
final_dataset_with_august = pd.concat([final_dataset_filtered, august_2019], ignore_index=True)

# Ordenamos el dataframe por 'customer_id', 'product_id' y 'periodo'
final_dataset_with_august = final_dataset_with_august.sort_values(by=['customer_id', 'product_id', 'periodo']).reset_index(drop=True)

final_dataset_filtered = final_dataset_with_august.copy()
# Mostramos los resultados
print("Dataset original con las ventas de agosto de 2019 calculadas:")
display(final_dataset_filtered)

Dataset original con las ventas de agosto de 2019 calculadas:


,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age
0,10001,20001,201701,0,11,99.43861,99.43861,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0,0
1,10001,20001,201702,0,23,198.84365,198.84365,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,2,0,1
2,10001,20001,201703,0,33,92.46537,92.46537,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,3,1,2
3,10001,20001,201704,0,8,13.29728,13.29728,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q2,4,0,3
4,10001,20001,201705,0,15,101.20711,101.00563,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q2,5,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320495,10580,20697,201903,0,1,0.00057,0.00057,PC,DEOS,RollOn,NIVEA,50,Aroma 11,Q1,3,1,26
1320496,10580,20697,201904,0,1,0.00058,0.00058,PC,DEOS,RollOn,NIVEA,50,Aroma 11,Q2,4,0,27
1320497,10580,20697,201906,0,1,0.00116,0.00116,PC,DEOS,RollOn,NIVEA,50,Aroma 11,Q2,6,1,29
1320498,10580,20697,201909,0,1,0.00058,0.00058,PC,DEOS,RollOn,NIVEA,50,Aroma 11,Q3,9,1,32


In [8]:
columns = ['plan_precios_cuidados', 'brand', 'sku_size', 'cat1', 'cat2', 'cat3', 'descripcion', 'quarter', 'month', 'close_quarter', 'tn'] #'cust_request_qty', 'cust_request_tn'
non_scalable_columns = ['cat1', 'cat2', 'cat3', 'brand', 'sku_size', 'plan_precios_cuidados', 'descripcion', 'quarter', 'month', 'close_quarter']
clusterizable_columns=['cat1', 'cat2', 'descripcion', 'brand']#, 'y']
n_features = len(columns)

Muetro cual es el primer mes de venta de cada producto, para debug (al mostrar el tail, ningun product/client deberia tener una primer venta en los ultimos 12 meses)

In [9]:
first_months = final_dataset_filtered.groupby(['product_id', 'customer_id'])['periodo'].min().reset_index()
first_months.columns = ['product_id', 'customer_id', 'first_month']

# Mostrar los resultados
print("Primer mes de cada producto:")
display(first_months.sort_values(by=['first_month', 'product_id', 'customer_id']).tail(150))

Primer mes de cada producto:


,product_id,customer_id,first_month
43971,20477,10278,201810
44014,20479,10021,201810
44435,20488,10018,201810
44552,20495,10004,201810
44556,20495,10012,201810
...,...,...,...
59416,21001,10018,201811
59661,21022,10003,201811
59922,21037,10007,201811
59923,21037,10018,201811


In [10]:
######
# Convierto las columnas categoricas a numericas
######
categorical_columns = ['cat1', 'cat2', 'cat3', 'brand', 'sku_size', 'descripcion', 'quarter']

# Transformar las columnas categóricas en numéricas
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    final_dataset_filtered[col] = le.fit_transform(final_dataset_filtered[col])
    label_encoders[col] = le
    
display(final_dataset_filtered)

,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age
0,10001,20001,201701,0,11,99.43861,99.43861,1,10,42,0,57,325,0,1,0,0
1,10001,20001,201702,0,23,198.84365,198.84365,1,10,42,0,57,325,0,2,0,1
2,10001,20001,201703,0,33,92.46537,92.46537,1,10,42,0,57,325,0,3,1,2
3,10001,20001,201704,0,8,13.29728,13.29728,1,10,42,0,57,325,1,4,0,3
4,10001,20001,201705,0,15,101.20711,101.00563,1,10,42,0,57,325,1,5,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320495,10580,20697,201903,0,1,0.00057,0.00057,2,3,59,23,15,24,0,3,1,26
1320496,10580,20697,201904,0,1,0.00058,0.00058,2,3,59,23,15,24,1,4,0,27
1320497,10580,20697,201906,0,1,0.00116,0.00116,2,3,59,23,15,24,1,6,1,29
1320498,10580,20697,201909,0,1,0.00058,0.00058,2,3,59,23,15,24,2,9,1,32


#### Aplico scalers

In [11]:
grouped = final_dataset_filtered.groupby(['customer_id', 'product_id'])


scalers = {col: {} for col in columns}
scaled_data_list = []

for (product_id, customer_id), group in grouped:
    # print(f"{product_id} - {customer_id}")
    scaled_group = group.copy()
    for col in columns:
        if col not in non_scalable_columns:
            scaler = StandardScaler()
            # print(col)
            scaled_group[col] = scaler.fit_transform(group[[col]])
            scalers[col][product_id] = {}
            scalers[col][product_id][customer_id] = scaler
    scaled_data_list.append(scaled_group)

# Combinar todos los datos escalados en un solo DataFrame
scaled_data = pd.concat(scaled_data_list)

In [12]:
display(scaled_data)

,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age
0,10001,20001,201701,0,11,99.43861,-0.735475,1,10,42,0,57,325,0,1,0,0
1,10001,20001,201702,0,23,198.84365,0.215317,1,10,42,0,57,325,0,2,0,1
2,10001,20001,201703,0,33,92.46537,-0.802173,1,10,42,0,57,325,0,3,1,2
3,10001,20001,201704,0,8,13.29728,-1.559402,1,10,42,0,57,325,1,4,0,3
4,10001,20001,201705,0,15,101.20711,-0.720487,1,10,42,0,57,325,1,5,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320495,10580,20697,201903,0,1,0.00057,-1.068568,2,3,59,23,15,24,0,3,1,26
1320496,10580,20697,201904,0,1,0.00058,-1.058433,2,3,59,23,15,24,1,4,0,27
1320497,10580,20697,201906,0,1,0.00116,-0.470626,2,3,59,23,15,24,1,6,1,29
1320498,10580,20697,201909,0,1,0.00058,-1.058433,2,3,59,23,15,24,2,9,1,32


#### DTW

In [13]:
from dtw import dtw

grouped_data = scaled_data.groupby(['product_id', 'customer_id'])

# Función para calcular DTW entre dos series
def calculate_dtw(series1, series2):
    # Calcular distancia DTW entre las series
    alignment = dtw(series1, series2, keep_internals=True)
    distance = alignment.distance
    return distance

# Diccionario para mantener el número de grupo
group_dict = {}
group_counter = 0

# Iterar sobre los grupos de datos
for (product_id, customer_id), group in grouped_data:
    if (product_id, customer_id) not in group_dict:
        # Crear un diccionario para almacenar las distancias a otras series
        distances = {}
        for (other_product_id, other_customer_id), other_group in grouped_data:
            if (other_product_id, other_customer_id) != (product_id, customer_id):
                # Calcular distancia DTW entre las series
                distance = calculate_dtw(group['tn'].values, other_group['tn'].values)
                distances[(other_product_id, other_customer_id)] = distance
        
        # Ordenar distancias y asignar grupos
        sorted_distances = sorted(distances.items(), key=lambda x: x[1])
        
        # Verificar si alguna serie cercana ya está asignada a un grupo
        assigned = False
        for (other_product_id, other_customer_id), _ in sorted_distances:
            if (other_product_id, other_customer_id) in group_dict:
                group_dict[(product_id, customer_id)] = group_dict[(other_product_id, other_customer_id)]
                assigned = True
                break
        
        # Si ninguna serie cercana tiene grupo asignado, crear un nuevo grupo
        if not assigned:
            group_dict[(product_id, customer_id)] = group_counter
            group_counter += 1

# Agregar la columna de grupo al dataframe original
scaled_data['group'] = scaled_data.apply(lambda row: group_dict[(row['product_id'], row['customer_id'])], axis=1)


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



KeyboardInterrupt: 

In [ ]:
display(scaled_data)

#### Funcion para preparar los datos y crear el modelo

El objetivo es predecir 2 dias en el futuro, por lo que la idea es re-armar el dataset. Donde el valor de X sera el conjunto de datos hasta N-2 e Y va a ser N (siendo N la cantidad de ventas para ese producto)

In [21]:
def prepare_train_data(data):
    X, y = []
    unique_combinations = data[['product_id', 'customer_id']].drop_duplicates()

    for _, row in unique_combinations.iterrows():
        product_id = row['product_id']
        customer_id = row['customer_id']
        product_customer_data = data[(data['product_id'] == product_id) & (data['customer_id'] == customer_id)].copy()
        product_customer_values = product_customer_data[columns].values
        
        X.append(product_customer_values[-(training_trashold):-2])
        y.append(product_customer_values[-1, -1])
    
    return np.array(X), np.array(y)

#### Agrupo los elemento por cluster y armo los modelos

In [ ]:
# Agrupar por 'group' y contar elementos distintos de 'product_id/customer_id'
grouped_clusters = scaled_data.groupby('group')

# Diccionario para almacenar el conteo de elementos distintos por grupo
unique_counts = {}

# Iterar sobre cada grupo de clusters
for group_number, group_data in grouped_clusters:
    # Crear una clave combinada de product_id y customer_id para contar elementos únicos
    group_data['combined_key'] = group_data['product_id'].astype(str) + '_' + group_data['customer_id'].astype(str)
    
    # Contar elementos únicos bajo la clave combinada
    unique_elements_count = group_data['combined_key'].nunique()
    
    # Almacenar el conteo en el diccionario
    unique_counts[group_number] = unique_elements_count

# Imprimir el resultado
for group_number, count in unique_counts.items():
    print(f"Grupo {group_number} tiene {count} combinaciones únicas de product_id/customer_id")

In [ ]:
from keras.optimizers import SGD, RMSprop, Adam
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, GRU
from keras.regularizers import l2
from sklearn.model_selection import KFold
import numpy as np
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

models = {}
predictions = {}

for cluster_number, cluster_data in grouped_clusters:
    print(f"Entrenando modelo para el cluster {cluster_number}")

    # Preparar los datos para el cluster actual
    X, y = prepare_train_data(cluster_data)

    if X.size == 0 or y.size == 0:
        print(f"Cluster {cluster_number} no tiene suficientes datos para entrenar.")
        continue


    # Crear el modelo LSTM
    model = Sequential()
    model.add(LSTM(128, activation='tanh', kernel_regularizer=l2(0.7), return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(0.1))
    model.add(LSTM(256, activation='tanh', kernel_regularizer=l2(0.7), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(0.7), return_sequences=True))
    model.add(Dropout(0.1))
    # model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(0.7), return_sequences=True))
    # model.add(Dropout(0.1))
    # model.add(LSTM(512, activation='tanh', kernel_regularizer=l2(0.7), return_sequences=True))
    # model.add(Dropout(0.1))
    model.add(LSTM(256, activation='tanh', kernel_regularizer=l2(0.7), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(128, activation='relu', kernel_regularizer=l2(0.7)))
    model.add(Dropout(0.1))
    model.add(Dense(1))

    # Compilar el modelo
    optimizer = Adam(learning_rate=0.01)
    model.compile(optimizer=optimizer, loss='mse')

    # Configurar callbacks
    callbacks = [
        ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=0.001),
        EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
    ]

    model.fit(X, y, epochs=150, verbose=2, batch_size=32, callbacks=callbacks)

    # Guardar el modelo en el diccionario por cluster
    models[cluster_number] = model

    # Imprimir el promedio de la métrica de evaluación (ej. MSE) de todas las particiones
    print(f"Modelo para el cluster {cluster_number} entrenado y guardado.")    


#### Predicciones a feb

In [ ]:
def prepare_kaggle_data(data):
    X_data = []
    unique_combinations = data[['product_id', 'customer_id']].drop_duplicates()

    for _, row in unique_combinations.iterrows():
        product_id = row['product_id']
        customer_id = row['customer_id']
        product_customer_data = data[(data['product_id'] == product_id) & (data['customer_id'] == customer_id)].copy()
        product_customer_values = product_customer_data[columns].values
        

        X_data.append((product_id, customer_id, product_customer_values[-treshold_months:]))
    
    return X_data

In [ ]:
predictions_dict = {}

# Prepara los datos
X_data = prepare_kaggle_data(scaled_data)

# Iterar sobre los productos con suficiente historia y hacer predicciones
for idx, product_values in X_data:
    product_id = product_values[0]['product_id']
    customer_id = product_values[0]['customer_id']
    
    # Obtener el cluster del producto/cliente
    product_cluster = scaled_data[(scaled_data['product_id'] == product_id) & (scaled_data['customer_id'] == customer_id)]['group'].values[0]
    
    # Obtener el modelo correspondiente al cluster
    model = models[product_cluster]
    
    # Preparar los datos de entrada para la predicción
    X = product_values.reshape(1, product_values.shape[0], product_values.shape[1])
    
    # Hacer la predicción
    prediction = model.predict(X)
    
    scaler = scalers['tn'][(product_id, customer_id)]
    
    # Guardar la predicción en el diccionario
    predictions_dict[(product_id, customer_id)] = scaler.inverse_transform([[prediction[0][0]]])[0][0]


In [ ]:
display(predictions_dict)

In [ ]:
predictions_dict_rounded = {key: round(prediction, 5) for key, prediction in predictions_dict.items()}
predictions_dict_df = pd.DataFrame(
    [(key[0], key[1], prediction) for key, prediction in predictions_dict_rounded.items()],
    columns=['product_id', 'customer_id', 'prediction']
)

predicciones.rename(columns={'promedio_tn': 'prediction'}, inplace=True)

# Unir el DataFrame predicciones con predictions_dict_df
combined_df = pd.concat([predicciones, predictions_dict_df], ignore_index=True)

# Sumarizar las predicciones por product_id
summarized_predictions_df = combined_df.groupby('product_id').agg({'prediction': 'sum'}).reset_index()

# Guardar el DataFrame resultante en un archivo CSV
summarized_predictions_df.to_csv('summarized_predictions.csv', index=False)

print("Archivo CSV 'summarized_predictions.csv' guardado correctamente.")